# Creating Custom Programs for Qiskit Runtime

<p>
<font size="4" color="#0f62fe">Paul Nation</font>
</p>
<p>
<font size="3" color="#0f62fe">IBM Quantum Partners Technical Enablement Team</font>
</p>

Here we will demonstrate how to create, upload, and use a custom Program for Qiskit Runtime.  As the utility of the Runtime execution engine lies in its ability to execute many quantum circuits with low latencies, this tutorial will show how to create your own Variational Quantum Eigensolver (VQE) program from scratch.

## Prerequisites

- You must have Qiskit 0.30+ installed.
- You must have an IBM Quantum account with the ability to upload a runtime program.  You have this ability if you belong to more than one provider.

## Current limitations

The runtime execution engine currently has the following limitations that must be kept in mind:

- The Docker images used by the runtime include only Qiskit and its dependencies, with few exceptions.  One exception is the inclusion of the `mthree` measurement mitigation package.


- For security reasons, the runtime cannot make internet calls outside of the environment.


- Your runtime program name must not contain an underscore`_`, otherwise it will cause an error when you try to execute it.

As Qiskit Runtime matures, these limitations will be removed.

## Simple VQE

VQE is an hybrid quantum-classical optimization procedure that finds the lowest eigenstate and eigenenergy of a linear system defined by a given Hamiltonian of Pauli operators.  For example, consider the following two-qubit Hamiltonian:


$$
H = A X_{1}\otimes X_{0} + A Y_{1}\otimes Y_{0} + A Z_{1}\otimes Z_{0},
$$

where $A$ is numerical coefficient and the subscripts label the qubits on which the operators act.  The zero index being farthest right is the ordering used in Qiskit.  The Pauli operators tell us which measurement basis to to use when measuring each of the qubits.

We want to find the ground state (lowest energy state) of this Hamiltonian, and the associated eigenvector.  To do this we must start at a given initial state and iteratively vary the parameters that define this state using a classical optimizer, such that the computed energies of subsequent steps are nominally lower than those previously.  The parameterized state of the system is defined by an ansatz quantum circuit that should have non-zero support in the direction of the ground state.  Because in general we do not know the solution, the choice of ansatz circuit can be highly problem-specific with a form dictated by additional information. For further information about variational algorithms, we point the reader to [Nature Reviews Physics volume 3, 625 (2021)](https://doi.org/10.1038/s42254-021-00348-9).


Thus we need at least the following inputs to create our VQE quantum program:

1. A representation of the Hamiltonian that specifies the problem.


2. A choice of parameterized ansatz circuit, and the ability to pass configuration options, if any.


However, the following are also beneficial inputs that users might want to have:

3. Add the ability to pass an initial state.


4. Vary the number of shots that are taken.


5. Ability to select which classical optimizer is used, and set configuraton values, if any. 


6. Ability to turn on and off measurement mitigation.


## Specifying the form of the input values

All inputs to runtime programs must be serializable objects.  That is to say, whatever you pass into a runtime program must be able to be converted to JSON format.  It is thus beneficial to keep inputs limited to basic data types and structures unless you have experience with custom object serialization, or they are common Qiskit types such as ``QuantumCircuit`` etc that the built-in `RuntimeEncoder` can handle.  Fortunately, the VQE program described above can be made out of simple Python components.

First, it is possible to represent any Hamiltonian using a list of values with each containing the numerical coefficeint for each term and the string representation for the Pauli operators.  For the above example, the ground state energy with $A=1$ is $-3$ and we can write it as:

In [1]:
H = [(1, 'XX'), (1, 'YY'), (1, 'ZZ')]

Next we have to provide the ability to specify the parameterized Ansatz circuit.  Here we will take advange of the fact that many ansatz circuits are pre-defined in the Qiskit Circuit Library.  Examples can be found in the [N-local circuits section](https://qiskit.org/documentation/apidoc/circuit_library.html#n-local-circuits).

We would like the user to be able to select between ansatz options such as: `NLocal`, `TwoLocal`, and `EfficientSU2`.  We could have the user pass the whole ansatz circuit to the program; however, in order to reduce the size of the upload we will pass the ansatz by name.  In the runtime program, we can take this name and get the class that it corresponds to from the library using, for example,  

In [2]:
import qiskit.circuit.library.n_local as lib_local

ansatz = getattr(lib_local, 'EfficientSU2')

For the ansatz configuration, we will pass a simple `dict` of values.

### Optionals 

- If we want to add the ability to pass an initial state, then we will need to add the ability to pass a 1D list/ NumPy array.  Because the number of parameters depends on the ansatz and its configuration, the user would have to know what ansatz they are doing ahead of time.


- Selecting a number of shots requires simply passing an integer value.


- Here we will allow selecting a classical optimizer by name from those in SciPy, and a `dict` of configuration parameters.  Note that for execution on an actual system, the noise inherent in today's quantum systems makes having a stochastic optimizer crucial to success.  SciPy does not have such a choice, and the one built into Qiskit is wrapped in such a manner as to make it difficult to use elsewhere.  As such, here we will use an SPSA optimizer written to match the style of those in SciPy.  This function is given in [Appendix A](#Appendix-A).

- Finally, for measurement error mitigation we can simply pass a boolean (True/False) value.

## Main program

We are now in a position to start building our main program.  However, before doing so we point out that it makes the code cleaner to make a separate fuction that takes strings of Pauli operators that define our Hamiltonian and convert them to a list of circuits with single-qubit gates that change the measurement basis for each qubit, if needed. This function is given in [Appendix B](#Appendix-B).

### Required signature

Every runtime program is defined via the `main` function, and must have the following input signature:

```
main(backend, user_message, *args, **kwargs)
```

where `backend` is the backend that the program is to be executed on, and `user_message` is the class by which interim (and possibly final) results are communicated back to the user.  After these two items, we add our program-specific arguments and keyword arguments.

### The main VQE program

Here is the main program for our sample VQE.  What each element of the function does is written in the comments before the element appears.

In [3]:
# Grab functions and modules from dependencies
import numpy as np
import scipy.optimize as opt
from scipy.optimize import OptimizeResult
import mthree

# Grab functions and modules from Qiskit needed
from qiskit import QuantumCircuit, transpile
import qiskit.circuit.library.n_local as lib_local

# The entrypoint for our Runtime Program
def main(backend, user_messenger,
         hamiltonian,
         ansatz='EfficientSU2',
         ansatz_config={},
         x0=None,
         optimizer='SPSA',
         optimizer_config={'maxiter': 100},
         shots = 8192,
         use_measurement_mitigation=False
        ):
    
    """
    The main sample VQE program.
    
    Parameters:
        backend (ProgramBackend): Qiskit backend instance.
        user_messenger (UserMessenger): Used to communicate with the
                                        program user.
        hamiltonian (list): Hamiltonian whose ground state we want to find.
        ansatz (str): Optional, name of ansatz quantum circuit to use,
                      default='EfficientSU2'
        ansatz_config (dict): Optional, configuration parameters for the
                              ansatz circuit.
        x0 (array_like): Optional, initial vector of parameters.
        optimizer (str): Optional, string specifying classical optimizer,
                         default='SPSA'.
        optimizer_config (dict): Optional, configuration parameters for the
                                 optimizer.
        shots (int): Optional, number of shots to take per circuit.
        use_measurement_mitigation (bool): Optional, use measurement mitigation,
                                           default=False.
        
    Returns:
        OptimizeResult: The result in SciPy optimization format.  
    """
    
    # Split the Hamiltonian into two arrays, one for coefficients, the other for
    # operator strings
    coeffs = np.array([item[0] for item in hamiltonian], dtype=complex)
    op_strings = [item[1] for item in hamiltonian]
    # The number of qubits needed is given by the number of elements in the strings
    # the defiune the Hamiltonian. Here we grab this data from the first element.
    num_qubits = len(op_strings[0])
    
    # We grab the requested ansatz circuit class from the Qiskit circuit library
    # n_local module and configure it using the number of qubits and options
    # passed in the ansatz_config.
    ansatz_instance = getattr(lib_local, ansatz)
    ansatz_circuit = ansatz_instance(num_qubits, **ansatz_config)
    
    # Here we use our convenence function from Appendix B to get measurement circuits
    # with the correct single-qubit rotation gates.
    meas_circs = opstr_to_meas_circ(op_strings)
    
    # When computing the expectation value for the energy, we need to know if we
    # evaluate a Z measurement or and identity measurement.  Here we take and X and Y
    # operator in the strings and convert it to a Z since we added the rotations
    # with the meas_circs.
    meas_strings = [string.replace('X', 'Z').replace('Y', 'Z') for string in op_strings]
    
    # Take the ansatz circuits, add the single-qubit measurement basis rotations from
    # meas_circs, and finally append the measurements themselves.
    full_circs = [ansatz_circuit.compose(mcirc).measure_all(inplace=False) for mcirc in meas_circs]
    
    # Get the number of parameters in the ansatz circuit.
    num_params = ansatz_circuit.num_parameters
    
    # Use a given initial state, if any, or do random initial state.
    if x0:
        x0 = np.asarray(x0, dtype=float)
        if x0.shape[0] != num_params:
            raise ValueError('Number of params in x0 ({}) does not match number \
                              of ansatz parameters ({})'. format(x0.shape[0],
                                                                 num_params))
    else:
        x0 = 2*np.pi*np.random.rand(num_params)
        
    # Because we are in general targeting a real quantum system, our circuits must be transpiled
    # to match the system topology and, hopefully, optimize them.
    # Here we will set the transpiler to the most optimal settings where 'sabre' layout and
    # routing are used, along with full O3 optimization.

    # This works around a bug in Qiskit where Sabre routing fails for simulators (Issue #7098)
    trans_dict = {}
    if not backend.configuration().simulator:
        trans_dict = {'layout_method': 'sabre', 'routing_method': 'sabre'}
    trans_circs = transpile(full_circs, backend, optimization_level=3, **trans_dict)
    
    # If using measurement mitigation we need to find out which physical qubits our transpiled
    # circuits actually measure, construct a mitigation object targeting our backend, and
    # finally calibrate our mitgation by running calibration circuits on the backend.
    if use_measurement_mitigation:
        maps = mthree.utils.final_measurement_mapping(trans_circs)
        mit = mthree.M3Mitigation(backend)
        mit.cals_from_system(maps)
    
    # Here we define a callback function that will stream the optimizer parameter vector
    # back to the user after each iteration.  This uses the `user_messenger` object.
    # Here we convert to a list so that the return is user readable locally, but
    # this is not required.
    def callback(xk):
        user_messenger.publish(list(xk))
    
    # This is the primary VQE function executed by the optimizer. This function takes the 
    # parameter vector as input and returns the energy evaluated using an ansatz circuit
    # bound with those parameters.
    def vqe_func(params):
        # Attach (bind) parameters in params vector to the transpiled circuits.
        bound_circs = [circ.bind_parameters(params) for circ in trans_circs]
        
        # Submit the job and get the resultant counts back
        counts = backend.run(bound_circs, shots=shots).result().get_counts()
        
        # If using measurement mitigation apply the correction and
        # compute expectation values from the resultant quasiprobabilities
        # using the measurement strings.
        if use_measurement_mitigation:
            quasi_collection = mit.apply_correction(counts, maps)
            expvals = quasi_collection.expval(meas_strings)
        # If not doing any mitigation just compute expectation values
        # from the raw counts using the measurement strings.
        # Since Qiskit does not have such functionality we use the convenence
        # function from the mthree mitigation module.
        else:
            expvals = mthree.utils.expval(counts, meas_strings)
        
        # The energy is computed by simply taking the product of the coefficients
        # and the computed expectation values and summing them. Here we also
        # take just the real part as the coefficients can possibly be complex,
        # but the energy (eigenvalue) of a Hamiltonian is always real.
        energy = np.sum(coeffs*expvals).real
        return energy
    
    # Here is where we actually perform the computation.  We begin by seeing what
    # optimization routine the user has requested, eg. SPSA verses SciPy ones,
    # and dispatch to the correct optimizer.  The selected optimizer starts at
    # x0 and calls 'vqe_func' everytime the optimizer needs to evaluate the cost
    # function.  The result is returned as a SciPy OptimizerResult object.
    # Additionally, after every iteration, we use the 'callback' function to
    # publish the interm results back to the user. This is important to do
    # so that if the Program terminates unexpectedly, the user can start where they
    # left off.
    
    # Since SPSA is not in SciPy need if statement
    if optimizer == 'SPSA':
        res = fmin_spsa(vqe_func, x0, args=(), **optimizer_config,
                        callback=callback)
    # All other SciPy optimizers here
    else:
        res = opt.minimize(vqe_func, x0, method=optimizer,
                           options=optimizer_config, callback=callback)
    # Return result. OptimizeResult is a subclass of dict.
    return res

## Local testing

<div class="alert alert-block alert-info">
<b>Important:</b> You need to execute the code blocks in Appendices A and B before continuing.
</div>

We can test whether our routine works by simply calling the `main` function with a backend instance, a `UserMessenger`, and sample arguments.

In [7]:
from qiskit.providers.ibmq.runtime import UserMessenger
msg = UserMessenger()

In [8]:
# Use the local Aer simulator
from qiskit import Aer
backend = Aer.get_backend('qasm_simulator')

In [9]:
# Execute the main routine for our simple two-qubit Hamiltonian H, and perform 5 iterations of the SPSA solver.
main(backend, msg, H, optimizer_config={'maxiter': 5})

[0.6809298743297729, 6.212947793338935, 2.5468059211919893, 1.786105003661796, 1.0611822357686909, 4.866944587894351, 5.825252230048505, 1.8013910705748435, 1.1414155678382845, 4.058318084439127, 5.819240170576276, 3.6822434254987035, -0.3402317359052001, 1.3158695204805178, 1.4128211024397963, 1.099923546703455]
[0.809595974177736, 6.3416138931868975, 2.418139821344026, 1.9147711035097592, 0.9325161359207278, 4.738278488046388, 5.696586130200542, 1.6727249707268803, 1.2700816676862476, 4.186984184287089, 5.690574070728314, 3.8109095253466667, -0.46889783575316324, 1.1872034206325546, 1.2841550025918331, 0.9712574468554919]
[0.708108562890604, 6.2401264818997655, 2.519627232631158, 2.016258514796891, 1.0340035472078597, 4.83976589933352, 5.798073541487674, 1.5712375594397483, 1.3715690789733797, 4.085496772999957, 5.589086659441182, 3.7094221140595347, -0.3674104244660312, 1.0857160093454226, 1.1826675913047011, 1.0727448581426238]
[1.1557826031713798, 5.792452441618989, 2.967301272911

     fun: -1.5185546875
 message: 'Optimization terminated successfully.'
    nfev: 10
     nit: 5
 success: True
       x: array([1.18173396, 5.8184038 , 2.99325263, 1.59453584, 1.45572623,
       5.3133913 , 5.37635086, 2.04486296, 1.84519448, 3.61187137,
       5.11546126, 3.23579671, 0.10621498, 0.66399333, 1.60439027,
       1.49446754])

Having executed the above, we see that there are 5 parameter arrays returned, one for each callback, along with the final optimization result.  The parameter arrays are the interim results, and the `UserMessenger` object prints these values to the cell output.  The output itself is the answer we obtained, expressed as a SciPy `OptimizerResult` object.

## Program metadata

Program metadata is essentially the docstring for a runtime program.  It describes overall program information such as the program `name`, `description`, `version`, and the `max_execution_time` the program is allowed to run, as well as details the inputs and the outputs the program expects.  At a bare minimum the values described above are required:

In [10]:
meta = {
  "name": "sample-vqe",
  "description": "A sample VQE program.",
  "max_execution_time": 100000,
  "version": "1.0",
}

It is important to set the `max_execution_time` high enough so that your program does not get terminated unexpectedly.  Additionally, one should make sure that interim results are sent back to the user so that, if something does happen, the user can start where they left off.

It is, however, good form to detail the parameters and return types, as well as interim results.  That being said, if making a runtime intended to be used by others, this information would also likely be mirrored in the signature of a function or class that the user would interact with directly; end users should not directly call runtime programs.  We will see why below.  Nevertheless, let us add to our metadata.  First, the `parameters` section details the inputs the user is able to pass:

In [11]:
meta["parameters"] = [
    {"name": "hamiltonian", "description": "Hamiltonian whose ground state we want to find.", "type": "list", "required": True},
    {"name": "ansatz", "description": "Name of ansatz quantum circuit to use, default='EfficientSU2'", "type": "str", "required": False},
    {"name": "ansatz_config", "description": "Configuration parameters for the ansatz circuit.", "type": "dict", "required": False},
    {"name": "x0", "description": "Initial vector of parameters.", "type": "ndarray", "required": False},
    {"name": "optimizer", "description": "Classical optimizer to use, default='SPSA'.", "type": "str", "required": False},
    {"name": "optimizer_config", "description": "Configuration parameters for the optimizer.", "type": "dict", "required": False},
    {"name": "shots", "description": "Number of shots to take per circuit.", "type": "int", "required": False},
    {"name": "use_measurement_mitigation", "description": "Use measurement mitigation, default=False.", "type": "bool", "required": False}
  ]

Next, the `return_values` section tells about the return types:

In [12]:
meta['return_values'] = [
    {"name": "result", "description": "Final result in SciPy optimizer format.", "type": "OptimizeResult"}
  ]

and finally let us specify what comes back when an interim result is returned:

In [13]:
meta["interim_results"] = [
    {"name": "params", "description": "Parameter vector at current optimization step", "type": "ndarray"},
  ]

## Uploading the program

We now have all the ingredients needed to upload our program.  To do so we need to collect all of our code in one file, here called `sample_vqe.py` for uploading.  This limitation will be removed in later versions of Qiskit Runtime.  Alternatively, if the entire code is contained within a single Jupyter notebook cell, this can be done using the magic function

```
%%writefile my_program.py
```

To actually upload the program we need to get a Provider from our IBM Quantum account:

In [14]:
from qiskit import IBMQ
IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [15]:
provider = IBMQ.get_provider(group='deployed')

### Program upload

The call to `program_upload` takes the target Python file as `data` and the metadata as inputs. **If you have already uploaded the program this will raise an error and you must delete it first to continue**.

In [17]:
program_id = provider.runtime.upload_program(data='sample_vqe.py', metadata=meta)
program_id

'sample-vqe'

Here the returned `program_id` is the same as the program `name` given in the metadata.  You cannot have more than one program with the same `name` and `program_id`.  The `program_id` is how you should reference your program.

### Program information

We can query the program for information and see that our metadata is correctly being attached:

In [18]:
prog = provider.runtime.program(program_id)
print(prog)

sample-vqe:
  Name: sample-vqe
  Description: A sample VQE program.
  Version: 1.0
  Creation date: 2021-10-06T16:35:05.000000
  Max execution time: 100000
  Input parameters:
    - hamiltonian:
      Description: Hamiltonian whose ground state we want to find.
      Type: list
      Required: True
    - ansatz:
      Description: Name of ansatz quantum circuit to use, default='EfficientSU2'
      Type: str
      Required: False
    - ansatz_config:
      Description: Configuration parameters for the ansatz circuit.
      Type: dict
      Required: False
    - x0:
      Description: Initial vector of parameters.
      Type: ndarray
      Required: False
    - optimizer:
      Description: Classical optimizer to use, default='SPSA'.
      Type: str
      Required: False
    - optimizer_config:
      Description: Configuration parameters for the optimizer.
      Type: dict
      Required: False
    - shots:
      Description: Number of shots to take per circuit.
      Type: int
      Req

### Deleting a program

If you make a mistake and need to delete and/or re-upload the program, you can run the following, passing the `program_id`:

In [16]:
#provider.runtime.delete_program(program_id)

## Running the program

### Specify parameters

To run the program we need to specify the `options` that are used in the runtime environment (not the program variables).  At present, only the `backend_name` is required.

In [19]:
backend = provider.backend.ibmq_qasm_simulator
options = {'backend_name': backend.name()}

The `inputs` dictionary is used to pass arguments to the `main` function itself.  For example:

In [20]:
inputs = {}
inputs['hamiltonian'] = H
inputs['optimizer_config']={'maxiter': 10}

### Execute the program

We now can execute the program and grab the result.

In [21]:
job = provider.runtime.run(program_id, options=options, inputs=inputs)

In [22]:
job.result()

{'fun': -1.7041015625,
 'x': array([5.48641333, 5.29481314, 1.94736362, 3.66657466, 1.43831538,
        3.45048577, 2.55682838, 5.17384303, 3.32676072, 1.98927237,
        1.45631183, 2.30776481, 0.747845  , 2.59320525, 1.88153096,
        1.83551349]),
 'nit': 10,
 'nfev': 20,
 'message': 'Optimization terminated successfully.',
 'success': True}

A few things need to be pointed out. First, we did not get back any interim results, and second, the return object is a plain dictionary.  This is because we did not listen for the return results, and we did not tell the job how to format the return result.

### Listening for interim results

To listen for interm results we need to pass a callback function to `provider.runtime.run` that stores the results.  The callback takes two arguments `job_id` and the returned data:

In [23]:
interm_results = []
def vqe_callback(job_id, data):
    interm_results.append(data)

Executing again we get:

In [24]:
job2 = provider.runtime.run(program_id, options=options, inputs=inputs, callback=vqe_callback)

In [25]:
job2.result()

{'fun': -1.51806640625,
 'x': array([3.97398119, 5.93481711, 1.48784258, 1.00201616, 3.86435761,
        2.2892097 , 4.99677643, 2.92384204, 2.28391397, 4.92760363,
        3.87555485, 2.46378704, 5.36551257, 5.23483398, 1.39751033,
        3.49051556]),
 'nit': 10,
 'nfev': 20,
 'message': 'Optimization terminated successfully.',
 'success': True}

In [26]:
print(interm_results)

[[4.250790609575883, 5.344832078651422, 1.604530561315992, 1.729114029453097, 3.6831803228305633, 1.6171159690574806, 5.555981735513656, 2.155046619871, 1.877278498652554, 4.760861746303351, 4.635800743123114, 2.916319399066078, 4.151890772551669, 4.998652553876574, 1.1866228828155627, 2.8806986175616807], [3.711960790326464, 5.883661897900841, 2.143360380565411, 1.1902842102036777, 4.2220101420799825, 2.1559457883068998, 5.017151916264237, 2.693876439120419, 2.416108317901973, 4.222031927053932, 4.096970923873695, 2.377489579816659, 4.690720591801088, 5.537482373125993, 1.725452702064982, 3.4195284368111], [3.881207420085515, 6.052908527659892, 1.9741137508063602, 1.3595308399627286, 4.391256771839034, 2.3251924180659507, 4.847905286505186, 2.86312306887947, 2.585354947661024, 4.391278556812983, 3.9277242941146437, 2.54673620957571, 4.859967221560139, 5.368235743366942, 1.556206072305931, 3.5887750665701508], [3.643404370520135, 5.8151054780945115, 1.73631070124098, 1.1217277903973484

### Formatting the returned results

In order to format the return results into the desired format, we need to specify a decoder.  This decoder must have a `decode` method that gets called to do the actual conversion.  In our case `OptimizeResult` is a simple sub-class of `dict` so the formatting is simple.

In [27]:
from qiskit.providers.ibmq.runtime import ResultDecoder
from scipy.optimize import OptimizeResult

class VQEResultDecoder(ResultDecoder):
    @classmethod
    def decode(cls, data):
        data = super().decode(data)  # This is required to preformat the data returned.
        return OptimizeResult(data)

We can then use this when returning the job result:

In [28]:
job3 = provider.runtime.run(program_id, options=options, inputs=inputs)
job3.result(decoder=VQEResultDecoder)

     fun: -1.446533203125
 message: 'Optimization terminated successfully.'
    nfev: 20
     nit: 10
 success: True
       x: array([6.20139194, 1.44487526, 4.80969906, 3.95169717, 6.61564694,
       4.5083935 , 3.3469375 , 4.46116919, 2.20733362, 2.76561795,
       5.57917989, 3.66721291, 5.94231287, 2.13323392, 4.79632746,
       5.51741218])

## Simplifying program execution with wrapping functions

While runtime programs are powerful and flexible, they are not the most friendly things to interact with.  Therefore, if your program is intended to be used by others, it is best to make wrapper functions and/or classes that simplify the user experience.  Moreover, such wrappers allow for validation of user inputs on the client side, which can quickly find errors that would otherwise be raised later during the execution process - something that might have taken hours waiting in queue to get to.

Here we will make two helper routines.  First, a job wrapper that allows us to attach and retrieve the interim results directly from the job object itself, as well as decodes for us so that the end user need not worry about formatting the results themselves.

In [29]:
class RuntimeJobWrapper():
    """A simple Job wrapper that attaches interm results directly to the job object itself
    in the `interm_results attribute` via the `_callback` function.
    """
    def __init__(self):
        self._job = None
        self._decoder = VQEResultDecoder
        self.interm_results = []
        
    def _callback(self, job_id, xk):
        """The callback function that attaches interm results:
        
        Parameters:
            job_id (str): The job ID.
            xk (array_like): A list or NumPy array to attach.
        """
        self.interm_results.append(xk)
        
    def __getattr__(self, attr):
        if attr == 'result':
            return self.result
        else:
            if attr in dir(self._job):
                return getattr(self._job, attr)
            raise AttributeError("Class does not have {}.".format(attr))
        
    def result(self):
        """Get the result of the job as a SciPy OptimizerResult object.
        
        This blocks until job is done, cancelled, or errors.
        
        Returns:
            OptimizerResult: A SciPy optimizer result object.
        """
        return self._job.result(decoder=self._decoder)

Next, we create the actual function we want users to call to execute our program.  To this function we will add a series of simple validation checks (not all checks will be done for simplicity), as well as use the job wrapper defined above to simply the output.

In [30]:
import qiskit.circuit.library.n_local as lib_local

def vqe_runner(backend, hamiltonian,
               ansatz='EfficientSU2', ansatz_config={},
               x0=None, optimizer='SPSA',
               optimizer_config={'maxiter': 100},
               shots = 8192,
               use_measurement_mitigation=False):
    
    """Routine that executes a given VQE problem via the sample-vqe program on the target backend.
    
    Parameters:
        backend (ProgramBackend): Qiskit backend instance.
        hamiltonian (list): Hamiltonian whose ground state we want to find.
        ansatz (str): Optional, name of ansatz quantum circuit to use, default='EfficientSU2'
        ansatz_config (dict): Optional, configuration parameters for the ansatz circuit.
        x0 (array_like): Optional, initial vector of parameters.
        optimizer (str): Optional, string specifying classical optimizer, default='SPSA'.
        optimizer_config (dict): Optional, configuration parameters for the optimizer.
        shots (int): Optional, number of shots to take per circuit.
        use_measurement_mitigation (bool): Optional, use measurement mitigation, default=False.
        
    Returns:
        OptimizeResult: The result in SciPy optimization format.  
    """
    options = {'backend_name': backend.name()}
    
    inputs = {}
    
    # Validate Hamiltonian is correct
    num_qubits = len(H[0][1])
    for idx, ham in enumerate(hamiltonian):
        if len(ham[1]) != num_qubits:
            raise ValueError('Number of qubits in Hamiltonian term {} does not match {}'.format(idx,
                                                                                                num_qubits))
    inputs['hamiltonian'] = hamiltonian
    
    # Validate ansatz is in the module
    ansatz_circ = getattr(lib_local, ansatz, None)
    if not ansatz_circ:
        raise ValueError('Ansatz {} not in n_local circuit library.'.format(ansatz))
        
    inputs['ansatz'] = ansatz
    inputs['ansatz_config'] = ansatz_config
    
    # If given x0, validate its length against num_params in ansatz:
    if x0:
        x0 = np.asarray(x0)
        ansatz_circ = ansatz_circ(num_qubits, **ansatz_config)
        num_params = ansatz_circ.num_parameters
        if x0.shape[0] != num_params:
            raise ValueError('Length of x0 {} does not match number of params in ansatz {}'.format(x0.shape[0],
                                                                                                   num_params))
    inputs['x0'] = x0
    
    # Set the rest of the inputs
    inputs['optimizer'] = optimizer
    inputs['optimizer_config'] = optimizer_config
    inputs['shots'] = shots
    inputs['use_measurement_mitigation'] = use_measurement_mitigation
    
    rt_job = RuntimeJobWrapper()
    job = provider.runtime.run('sample-vqe', options=options, inputs=inputs, callback=rt_job._callback)
    rt_job._job = job
    
    return rt_job
    

We can now execute our runtime program via this runner function:

In [31]:
job4 = vqe_runner(backend, H, optimizer_config={'maxiter': 15})

In [32]:
job4.result()

     fun: -0.9384765625
 message: 'Optimization terminated successfully.'
    nfev: 30
     nit: 15
 success: True
       x: array([ 4.51907826,  5.27223811,  3.91595188,  2.96827229,  4.72345222,
       -1.50412337,  1.70661237,  3.43154   ,  4.88147756,  1.34614751,
        2.59952558,  0.13369082,  1.32651411,  5.801332  ,  4.4509297 ,
        3.44059551])

The interim results are now attached to the job `interm_results` attribute and, as expected, we see that the length matches the number of iterations performed.

In [33]:
len(job4.interm_results)

15

## Conclusion

We have demonstrated how to create, upload, and use a custom Qiskit Runtime by creating our own VQE solver from scratch.  This tutorial was meant to touch upon every aspect of the process for a real-world example.  Within the current limitations of the runtime environment, this example should enable readers to develop their own single-file runtime program.  This program is also a good starting point for exploring additional flavors of VQE runtime.  For example, it is straightforward to vary the number of shots per iteration, increasing shots as the number of iterations increases.  Those looking to go deeper can consider implimenting an [adaptive VQE](https://doi.org/10.1038/s41467-019-10988-2), where the ansatz is not fixed at initialization.

## Appendix A

Here we code a simple simultaneous perturbation stochastic approximation (SPSA) optimizer for use on noisy quantum systems.  Most optimizers do not handle fluctuating cost functions well, so this is a needed addition for executing on real quantum hardware.

In [4]:
import numpy as np
from scipy.optimize import OptimizeResult

def fmin_spsa(func, x0, args=(), maxiter=100,
              a=1.0, alpha=0.602, c=1.0, gamma=0.101,
              callback=None):
    """
    Minimization of scalar function of one or more variables using simultaneous
    perturbation stochastic approximation (SPSA).

    Parameters:
        func (callable): The objective function to be minimized.

                          ``fun(x, *args) -> float``

                          where x is an 1-D array with shape (n,) and args is a
                          tuple of the fixed parameters needed to completely 
                          specify the function.

        x0 (ndarray): Initial guess. Array of real elements of size (n,), 
                      where ‘n’ is the number of independent variables.
      
        maxiter (int): Maximum number of iterations.  The number of function
                       evaluations is twice as many. Optional.

        a (float): SPSA gradient scaling parameter. Optional.

        alpha (float): SPSA gradient scaling exponent. Optional.

        c (float):  SPSA step size scaling parameter. Optional.
        
        gamma (float): SPSA step size scaling exponent. Optional.

        callback (callable): Function that accepts the current parameter vector
                             as input.

    Returns:
        OptimizeResult: Solution in SciPy Optimization format.

    Notes:
        See the `SPSA homepage <https://www.jhuapl.edu/SPSA/>`_ for usage and
        additional extentions to the basic version implimented here.
    """
    A = 0.01 * maxiter
    x0 = np.asarray(x0)
    x = x0

    for kk in range(maxiter):
        ak = a*(kk+1.0+A)**-alpha
        ck = c*(kk+1.0)**-gamma
        # Bernoulli distribution for randoms
        deltak =  2*np.random.randint(2, size=x.shape[0])-1
        grad = (func(x + ck*deltak, *args) - func(x - ck*deltak, *args))/(2*ck*deltak)
        x -= ak*grad
        
        if callback is not None:
            callback(x)

    return OptimizeResult(fun=func(x, *args), x=x, nit=maxiter, nfev=2*maxiter, 
                          message='Optimization terminated successfully.',
                          success=True)

## Appendix B

This is a helper function that converts the Pauli operators in the strings that define the Hamiltonian operators into the appropriate measurements at the end of the circuits.  For $X$ operators this involves adding an $H$ gate to the qubits to be measured, whereas a $Y$ operator needs $S^{+}$ followed by a $H$.  Other choices of Pauli operators require no additional gates prior to measurement.

In [5]:
def opstr_to_meas_circ(op_str):
    """Takes a list of operator strings and makes circuit with the correct post-rotations for measurements.

    Parameters:
        op_str (list): List of strings representing the operators needed for measurements.

    Returns:
        list: List of circuits for measurement post-rotations
    """
    num_qubits = len(op_str[0])
    circs = []
    for op in op_str:
        qc = QuantumCircuit(num_qubits)
        for idx, item in enumerate(op):
            if item == 'X':
                qc.h(num_qubits-idx-1)
            elif item == 'Y':
                qc.sdg(num_qubits-idx-1)
                qc.h(num_qubits-idx-1)
        circs.append(qc)
    return circs

In [6]:
from qiskit.tools.jupyter import *
%qiskit_copyright